# Biblioteca

In [1]:
import pandas as pd
import numpy as np

import requests

# Chave API, sempre tente colocar num arquivo para ler

In [2]:
key = open('riot_api_key.txt', "r").read()

### Coleta dos ids dos jogadores

In [3]:
region = 'br1'
elo = 'challenger'

highelo_summoner_ids = []

url_id = ('https://' + region + '.api.riotgames.com/lol/league/v4/' + elo 
          + 'leagues/by-queue/RANKED_SOLO_5x5' + '/?api_key=' + key)
response = requests.get(url_id)

for i in range(0, len(response.json()['entries'])):
    highelo_summoner_ids.append(response.json()['entries'][i]['summonerId'])

Aqui eu coletei todos os jogadores do challenger brasileiro. Vamos escolher apenas um para poder gerar o puuid (somente assim consigo acessar os dados de partidas). Assim, vamos coletar 20 partidas desse jogador.

### PUUID

In [4]:
summoner_id = highelo_summoner_ids[0]

url_puuid = ('https://' + region + '.api.riotgames.com/lol/summoner/v4/summoners/'
             + summoner_id + '/?api_key=' + key)
response = requests.get(url_puuid)
summoner_puuid = response.json()['puuid']

### Coletando os ids das partidas

In [5]:
match_ids = np.array([])

url_match_id = ('https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/' 
                 + summoner_puuid + '/ids?queue=420&api_key=' + key)
response = requests.get(url_match_id)
match_ids = np.append(match_ids, response.json())

### Coletando os dados das partidas

In [6]:
matches = []

for match_id in match_ids:
    url_match = ('https://americas.api.riotgames.com/lol/match/v5/matches/' 
                 + match_id + '/?api_key=' + key)
    response = requests.get(url_match)
    matches = np.append(matches, response.json())

### Variáveis de interesse e renomeação das colunas

In [7]:
interested_var = ['assists', 'baronKills', 'champLevel', 'damageDealtToObjectives', 
                  'detectorWardsPlaced', 'dragonKills', 'firstBloodKill',
                  'firstTowerKill', 'gameEndedInSurrender', 'goldEarned',
                  'kills', 'pentaKills', 'teamId', 
                  'totalDamageDealtToChampions',  'totalHeal', 'totalHealsOnTeammates',
                  'totalMinionsKilled', 'visionScore', 'wardsKilled',
                  'wardsPlaced', 'win']

b_column_names = {'assists': 'bTotalAssists',
                  'baronKills': 'bTotalBaronKills',
                  'champLevel': 'bSumChampLevel',
                  'damageDealtToObjectives': 'bTotalDamageObjectives',
                  'detectorWardsPlaced': 'bTotalPinkWardPlaced',
                  'dragonKills': 'bTotalDragonKills',
                  'firstBloodKill': 'bFirstBlood',
                  'firstTowerKill': 'bFirstTower',
                  'gameEndedInSurrender': 'bSurrender',
                  'goldEarned': 'bTotalGold',
                  'kills': 'bTotalKills',
                  'pentaKills': 'bCountPentaKills',
                  'totalDamageDealtToChampions': 'bTotalDamageEnemy',
                  'totalHeal': 'bTotalHeal',
                  'totalHealsOnTeammates': 'bTotalHealOnTeammates',
                  'totalMinionsKilled': 'bTotalMinionsKilled',
                  'visionScore': 'bVisionScore',
                  'wardsKilled': 'bWardsKilled',
                  'wardsPlaced': 'bTotalWardsPlaced',
                  'win': 'bWin'}

r_column_names = {'assists': 'rTotalAssists',
                  'baronKills': 'rTotalBaronKills',
                  'champLevel': 'rSumChampLevel',
                  'damageDealtToObjectives': 'rTotalDamageObjectives',
                  'detectorWardsPlaced': 'rTotalPinkWardPlaced',
                  'dragonKills': 'rTotalDragonKills',
                  'firstBloodKill': 'rFirstBlood',
                  'firstTowerKill': 'rFirstTower',
                  'gameEndedInSurrender': 'rSurrender',
                  'goldEarned': 'rTotalGold',
                  'kills': 'rTotalKills',
                  'pentaKills': 'rCountPentaKills',
                  'totalDamageDealtToChampions': 'rTotalDamageEnemy',
                  'totalHeal': 'rTotalHeal',
                  'totalHealsOnTeammates': 'rTotalHealOnTeammates',
                  'totalMinionsKilled': 'rTotalMinionsKilled',
                  'visionScore': 'rVisionScore',
                  'wardsKilled': 'rWardsKilled',
                  'wardsPlaced': 'rTotalWardsPlaced',
                  'win': 'rWin'}

### Processamento dos dados

In [10]:
df_matches = pd.DataFrame()
for match in matches:
    df_temp = pd.DataFrame(columns = interested_var)
    if match['info']['gameVersion'][:2] == '11':
        duration = match['info']['gameDuration']
        if duration > 900:
            participants = match['info']['participants']
            list_summoners = []
            for player in participants:
                summoner = {key : player[key] for key in interested_var}
                
                list_summoners.append(summoner)
            df_temp = df_temp.append(list_summoners, ignore_index = True)
            df_temp = df_temp.groupby('teamId').sum()
            b_team = df_temp.loc[[100]].rename(columns = b_column_names).reset_index(drop = True)
            r_team = df_temp.loc[[200]].rename(columns = r_column_names).reset_index(drop = True)
            df_match = pd.concat([b_team, r_team], axis = 1)
            df_match['GameDuration'] = duration
            df_match['bAvgChampLevel'] = df_match['bSumChampLevel'] / 5
            df_match['rAvgChampLevel'] = df_match['rSumChampLevel'] / 5
            df_match['bWin'] = df_match['bWin'] / 5
            df_match['rWin'] = df_match['rWin'] / 5
            df_matches = df_matches.append(df_match)
df_matches = df_matches.reset_index(drop = True)

### Resultado

In [11]:
df_matches.to_csv("df_matches_exemplo.csv", index = False)
df_matches

,bTotalAssists,bTotalBaronKills,bSumChampLevel,bTotalDamageObjectives,bTotalPinkWardPlaced,bTotalDragonKills,bFirstBlood,bFirstTower,bSurrender,bTotalGold,...,rTotalHeal,rTotalHealOnTeammates,rTotalMinionsKilled,rVisionScore,rWardsKilled,rTotalWardsPlaced,rWin,GameDuration,bAvgChampLevel,rAvgChampLevel
0,48,1,74,61060,21,1,0,0,0,56942,...,12939,139,596,170,41,73,0.0,1565,14.8,13.2
1,16,0,48,5627,14,0,1,0,5,27119,...,16823,653,463,69,13,35,1.0,1004,9.6,10.6
2,52,1,72,84246,21,2,0,0,0,56278,...,11508,0,594,128,15,50,0.0,1471,14.4,12.0
3,40,0,66,7896,13,0,1,0,5,47315,...,34901,0,542,142,16,47,1.0,1618,13.2,15.4
4,28,0,63,26311,5,1,1,0,5,43876,...,21915,4086,559,114,11,55,1.0,1421,12.6,13.8
5,44,1,81,67889,35,2,1,1,5,67326,...,49699,9101,870,258,47,110,1.0,2206,16.2,16.8
6,22,0,55,8167,11,0,0,0,0,33798,...,21361,4544,508,100,11,37,1.0,1290,11.0,12.6
7,32,0,76,43740,31,2,0,0,0,58736,...,26973,1011,835,212,39,77,1.0,1885,15.2,16.8
8,82,1,86,66003,30,2,0,0,0,76567,...,46523,789,770,227,45,84,0.0,2316,17.2,17.2
9,36,1,70,69585,10,1,0,0,0,53255,...,20726,4306,591,140,19,77,0.0,1503,14.0,13.0
